In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import pandas as pd
import numpy as np 

In [0]:
df_raw = pd.read_csv('/content/drive/My Drive/data cleaning with python and pandas/property data.csv')

In [4]:
df_raw.head()

,PID,ST_NUM,ST_NAME,OWN_OCCUPIED,NUM_BEDROOMS,NUM_BATH,SQ_FT
0,100001000.0,104.0,PUTNAM,Y,3,1,1000
1,100002000.0,197.0,LEXINGTON,N,3,1.5,--
2,100003000.0,NaN,LEXINGTON,N,NaN,1,850
3,100004000.0,201.0,BERKELEY,12,1,NaN,700
4,NaN,203.0,BERKELEY,Y,3,2,1600


In [0]:
df_raw.drop('PID', axis=1, inplace=True)

**DATA Information:**


*   ST_NUM: Street number

*   ST_NAME: Street name

*   OWN_OCCUPIED: Is the residence owner occupied

*   NUM_BEDROOMS: Number of bedrooms




In [6]:
df_raw['NUM_BEDROOMS'].head(15)

0      3
1      3
2    NaN
3      1
4      3
5    NaN
6      2
7      1
8     na
Name: NUM_BEDROOMS, dtype: object

Here we can see that this column has some non-standard NaN values.

In [0]:
missing_values = ['na', 'n/a', '--']
df_train = pd.read_csv('/content/drive/My Drive/data cleaning with python and pandas/property data.csv', na_values=missing_values)

In [9]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9 entries, 0 to 8
Data columns (total 7 columns):
PID             8 non-null float64
ST_NUM          7 non-null float64
ST_NAME         9 non-null object
OWN_OCCUPIED    8 non-null object
NUM_BEDROOMS    6 non-null float64
NUM_BATH        8 non-null object
SQ_FT           7 non-null float64
dtypes: float64(4), object(3)
memory usage: 584.0+ bytes


##Unexpected ssing Values

In [10]:
df_train['OWN_OCCUPIED'].values

array(['Y', 'N', 'N', '12', 'Y', 'Y', nan, 'Y', 'Y'], dtype=object)

Here we see a value of **'12'**, which is unexpected. We have to deal with this problem.

This example is a little more complicated so we’ll need to think through a strategy for detecting these types of missing values. There’s a number of different approaches, but here’s the way that I’m going to work through this one:


*   Loop through the OWN_OCCUPIED column.

*   Try and turn the entry into an integer.

*   If the entry can be changed into an integer, enter a missing value.

*   If the number can’t be an integer, we know it’s a string, so keep going.



In [0]:
count = 0

for row in df_train['OWN_OCCUPIED']:
  try:
    int(row)
    df_train.loc[count, 'OWN_OCCUPIED'] = np.NaN
  except ValueError:
    pass
  count+=1

In [14]:
df_train.isnull().sum()

PID             1
ST_NUM          2
ST_NAME         0
OWN_OCCUPIED    2
NUM_BEDROOMS    3
NUM_BATH        1
SQ_FT           2
dtype: int64

The column **OWN_OCCUPIED** has 2 null values rahter than one from before.

Now that we’ve summarized the number of missing values, let’s take a look at doing some simple replacements.

##Imputing the missing values

In [0]:
num_cols = [x for x in df_train.columns if df_train[x].dtype == 'float64']

In [0]:
cat_cols = [x for x in df_train.columns if x not in num_cols]

In [20]:
print(num_cols)
print(cat_cols)

['PID', 'ST_NUM', 'NUM_BEDROOMS', 'SQ_FT']
['ST_NAME', 'OWN_OCCUPIED', 'NUM_BATH']


In [0]:
from sklearn.impute import SimpleImputer

for x in num_cols:
  median = df_train[x].median()
  df_train[x].fillna(median, inplace=True)

for x in cat_cols:
  df_train[x].fillna('NaN', inplace=True)

In [27]:
df_train.isnull().sum()

PID             0
ST_NUM          0
ST_NAME         0
OWN_OCCUPIED    0
NUM_BEDROOMS    0
NUM_BATH        0
SQ_FT           0
dtype: int64

Here for categorical columns I have converted NaN values with string NaN. There are many ways to replace the missing values. This was one of them. We can replace values according to our problem in hand.